In [1]:
import numpy as np
from cfd_simulate import simulate
from three_desc_model import FingerprintCalculator, NeighborlistCalculator





ModuleNotFoundError: No module named 'cfd_simulate'

In [ ]:

# Function to simulate and get energy output
def run_cfd_simulation(positions):
    result = simulate(positions, sim_number=0)
    # Assume result contains energy output or modify based on actual simulation output
    return result['energy_output']

# Function to calculate interference using FingerprintCalculator
def calculate_interference(positions):
    neighbor_calculator = NeighborlistCalculator(cutoff=279.0, cone_grad=0.3, cone_offset=100.0)
    neighbors = neighbor_calculator.calculate(turbinelist=range(len(positions)), turbineposition=positions)
    fingerprint_calculator = FingerprintCalculator(cutoff=279.0, Gs={'turbine': [{'type': 'G2', 'eta': 0.1}]}, Rct=100.0, delta_R=0.1, cone_grad=0.3, cone_offset=100.0)
    fingerprints = fingerprint_calculator.calculate(turbinelist=range(len(positions)), turbineposition=positions, neighborlist=neighbors, symbol='turbine')
    interference = sum([np.mean(fp) for fp in fingerprints])
    return interference

# Objective function combining energy output and interference penalty
def objective_function(positions, lambda_):
    energy_output = run_cfd_simulation(positions)
    interference_penalty = calculate_interference(positions)
    return energy_output - lambda_ * interference_penalty

# Gradient calculation (numerical approximation)
def compute_gradients(positions, lambda_):
    epsilon = 1e-5
    gradients = np.zeros_like(positions)
    for i in range(len(positions)):
        for j in range(2):  # x and y coordinates
            positions[i, j] += epsilon
            f_plus = objective_function(positions, lambda_)
            positions[i, j] -= 2 * epsilon
            f_minus = objective_function(positions, lambda_)
            gradients[i, j] = (f_plus - f_minus) / (2 * epsilon)
            positions[i, j] += epsilon
    return gradients

# Gradient Descent Algorithm
def gradient_descent(initial_positions, learning_rate, lambda_, num_iterations):
    positions = initial_positions
    for i in range(num_iterations):
        gradients = compute_gradients(positions, lambda_)
        positions = positions - learning_rate * gradients
        positions = np.clip(positions, 0, [4000, 4000])  # Adjust according to the actual farm size
        if np.linalg.norm(gradients) < 1e-6:  # Convergence criteria
            break
        print(f"Iteration {i+1}, Objective Function Value: {objective_function(positions, lambda_)}")
    return positions

# Example initial positions
def generate_random_positions(num_turbines, farm_width, farm_height):
    return np.random.rand(num_turbines, 2) * [farm_width, farm_height]

In [ ]:
# Parameters
initial_positions = generate_random_positions(num_turbines=5, farm_width=4000, farm_height=4000)
learning_rate = 0.01
lambda_ = 0.5
num_iterations = 100

# Run gradient descent
optimized_positions = gradient_descent(initial_positions, learning_rate, lambda_, num_iterations)
print("Optimized Positions:", optimized_positions)